<a href="https://colab.research.google.com/github/VishalPrem1994/AIGenPlayGround/blob/main/TFAutoModelForTokenClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install transformers datasets torch tochvision
!pip install accelerate -U

ERROR: Could not find a version that satisfies the requirement tochvision (from versions: none)
ERROR: No matching distribution found for tochvision


In [8]:
from transformers import AutoProcessor, TFAutoModelForTokenClassification, Trainer
import accelerate
import transformers
from datasets import load_dataset
import torch

### Inference

In [3]:


processor = AutoProcessor.from_pretrained("microsoft/layoutlmv3-base", apply_ocr=False)
model = TFAutoModelForTokenClassification.from_pretrained("microsoft/layoutlmv3-base", num_labels=7)

dataset = load_dataset("nielsr/funsd-layoutlmv3", split="test")
example = dataset[0]
image = example["image"]
words = example["tokens"]
boxes = example["bboxes"]
word_labels = example["ner_tags"]

encoding = processor(image, words, boxes=boxes, word_labels=word_labels, return_tensors="tf")

outputs = model(**encoding)
# loss = outputs.loss
# logits = outputs.logits

All model checkpoint layers were used when initializing TFLayoutLMv3ForTokenClassification.

Some layers of TFLayoutLMv3ForTokenClassification were not initialized from the model checkpoint at microsoft/layoutlmv3-base and are newly initialized: ['dropout', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [4]:
print(model.num_labels)

7


In [5]:
outputs.logits.numpy()

array([[[ 0.50071174,  0.01239661,  0.43814945, ..., -0.61552405,
         -0.1330928 ,  0.39383796],
        [ 0.10474317,  0.00301229, -0.07573045, ..., -0.40868038,
         -0.15381873, -0.29832837],
        [ 0.21187952,  0.01025306,  0.24905418, ..., -0.603045  ,
         -0.31270793,  0.08801183],
        ...,
        [-0.16728249,  0.04141575,  0.19596562, ..., -0.46404114,
          0.05456414,  0.19827469],
        [ 0.1220977 , -0.16585322, -0.42728674, ..., -0.40467927,
          0.16650699,  0.7635721 ],
        [ 0.50325793,  0.01232533,  0.43766943, ..., -0.6147721 ,
         -0.13373575,  0.39446604]]], dtype=float32)

In [9]:
torch.softmax(torch.tensor(outputs.logits.numpy()),dim=-1)

tensor([[[0.1947, 0.1195, 0.1829,  ..., 0.0638, 0.1033, 0.1750],
         [0.1705, 0.1540, 0.1423,  ..., 0.1020, 0.1316, 0.1139],
         [0.1750, 0.1431, 0.1817,  ..., 0.0775, 0.1036, 0.1546],
         ...,
         [0.1170, 0.1441, 0.1682,  ..., 0.0870, 0.1460, 0.1686],
         [0.1416, 0.1062, 0.0818,  ..., 0.0836, 0.1481, 0.2690],
         [0.1949, 0.1193, 0.1826,  ..., 0.0637, 0.1031, 0.1748]]])

### training

In [ ]:
!pip install transformers datasets
!pip install transformers -U
!pip install accelerate -U

In [20]:
from transformers import AutoProcessor, TFAutoModelForTokenClassification, Trainer, TrainingArguments
from datasets import load_dataset, Image
from PIL import Image as mg
from datasets import Features, Sequence, ClassLabel, Value, Array2D, Array3D
import accelerate
import transformers

In [11]:
datasets = load_dataset("nielsr/funsd")
datasets

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'words', 'bboxes', 'ner_tags', 'image_path'],
        num_rows: 149
    })
    test: Dataset({
        features: ['id', 'words', 'bboxes', 'ner_tags', 'image_path'],
        num_rows: 50
    })
})

In [12]:
labels = datasets['train'].features['ner_tags'].feature.names
print(labels)

['O', 'B-HEADER', 'I-HEADER', 'B-QUESTION', 'I-QUESTION', 'B-ANSWER', 'I-ANSWER']


In [13]:
id2label = {v: k for v, k in enumerate(labels)}
label2id = {k: v for v, k in enumerate(labels)}
label2id

{'O': 0,
 'B-HEADER': 1,
 'I-HEADER': 2,
 'B-QUESTION': 3,
 'I-QUESTION': 4,
 'B-ANSWER': 5,
 'I-ANSWER': 6}

In [18]:
features = Features({
    'image': Array3D(dtype="int64", shape=(3, 224, 224)),
    'input_ids': Sequence(feature=Value(dtype='int64')),
    'attention_mask': Sequence(Value(dtype='int64')),
    'token_type_ids': Sequence(Value(dtype='int64')),
    'bbox': Array2D(dtype="int64", shape=(512, 4)),
    'labels': Sequence(ClassLabel(names=labels)),
})

In [ ]:
processor = AutoProcessor.from_pretrained("microsoft/layoutlmv3-base", apply_ocr=False)

def preprocess_data(examples):
  images = [mg.open(path).convert("RGB") for path in examples['image_path']]
  words = examples['words']
  boxes = examples['bboxes']
  word_labels = examples['ner_tags']

  encoded_inputs = processor(images, words, boxes=boxes, word_labels=word_labels,padding="max_length", truncation=True, return_tensors="tf")

  return encoded_inputs

train_dataset = datasets['train'].map(preprocess_data, batched=True, remove_columns=datasets['train'].column_names,
                                      features=features)
test_dataset = datasets['test'].map(preprocess_data, batched=True, remove_columns=datasets['test'].column_names,
                                      features=features)

Map:   0%|          | 0/149 [00:00<?, ? examples/s]

In [ ]:
model = TFAutoModelForTokenClassification.from_pretrained("microsoft/layoutlmv3-base", num_labels=7)

In [ ]:
args = TrainingArguments(
    output_dir="layoutlmv2-finetuned-funsd-v2", # name of directory to store the checkpoints
    max_steps=1000, # we train for a maximum of 1,000 batches
    warmup_ratio=0.1, # we warmup a bit
    fp16=True, # we use mixed precision (less memory consumption)
    push_to_hub=False, # after training, we'd like to push our model to the hub
)

In [ ]:
trainer = Trainer(model=model,args=None,train_dataset=None,eval_dataset=None)

In [ ]:
processor = AutoProcessor.from_pretrained("microsoft/layoutlmv3-base", apply_ocr=False)
model = TFAutoModelForTokenClassification.from_pretrained("microsoft/layoutlmv3-base", num_labels=7)

dataset = load_dataset("nielsr/funsd-layoutlmv", split="test")
example = dataset[0]
image = example["image"]
words = example["tokens"]
boxes = example["bboxes"]
word_labels = example["ner_tags"]

encoding = processor(image, words, boxes=boxes, word_labels=word_labels, return_tensors="tf")

outputs = model(**encoding)
# loss = outputs.loss
# logits = outputs.logits

In [ ]:
from PIL import Image
dataset = load_dataset("nielsr/funsd-layoutlmv3", split="test")
example = dataset[0]
image = example["image"]
image.copy()

In [ ]:

torch.softmax(torch.tensor(outputs.logits.numpy()),dim=-1)